In [1]:
import pickle
from tqdm.notebook import tqdm
import os


In [2]:
folder = '/scratche/home/apoorv/transformer-kgc/data'

In [3]:
subgraph_split = 'MetaQA'

In [4]:
def readTriples(fname):
    f = open(fname)
    triples = []
    for line in f:
        if line.startswith('predict head:'):
            continue
        line = line.rstrip()
        line = line.replace('predict tail: ', '')
        line = line.split('\t')
        head_rel = line[0]
        tail = line[1]
        head_rel = head_rel.replace(' | ', '|')
        head_rel = head_rel.replace(' |', '|')
        head, rel, _ = head_rel.split('|')
        if rel != 'noop':
            triples.append([head, rel, tail])
    f.close()
    return triples

def getEntities(triples):
    ents = set()
    for t in triples:
        ents.add(t[0])
        ents.add(t[2])
    return ents

def readLines(fname):
    f = open(fname)
    lines = []
    for line in f:
        lines.append(line.rstrip())
    return lines

In [5]:
def getHeadFromQuestion(q):
    h1 = q.split('[')[1]
    head = h1.split(']')[0]
    return head

def readQuestions(fname):
    f = open(fname)
    head_answer_pairs = []
    for line in f:
        q, answer_string = line.rstrip().split('\t')
        head = getHeadFromQuestion(q)
        answers = answer_string.split('|')
        head_answer_pairs.append((head, answers))
    return head_answer_pairs
        

In [8]:
q_file_name = os.path.join('data/MetaQA_half/', 'qa_train_2hop.txt')
ha_pairs = readQuestions(q_file_name)

In [10]:
folder

'/scratche/home/apoorv/transformer-kgc/data'

In [12]:
from unidecode import unidecode
import networkx as nx
# subgraph_splits = ['MetaQA_half', 'MetaQA_half2', 'MetaQA_half3', 'MetaQA_half_allents',
#                   'MetaQA_half_allents2', 'MetaQA_half_allents3']
subgraph_splits = ['MetaQA']
for sg in tqdm(subgraph_splits):
    fname = os.path.join(folder, sg, 'train.txt')
    train = readTriples(fname)
#     print(len(train))
    ents = getEntities(train)
    G = nx.Graph()
    G.add_nodes_from(ents)
    for t in train:
        G.add_edge(t[0], t[2])
#     print(len(G.edges))
    count = 0
    for ha_pair in ha_pairs:
        head = ha_pair[0]
        answers = ha_pair[1]
        head = unidecode(head).rstrip()
        if head not in G:
            continue
        
        head_nbhood = set(G.neighbors(head))
        
        flag = 0
        for a in answers:
            if a in head_nbhood:
                count += 1
                flag = 1
                break
#         if len(set(G.neighbors(head)).intersection(set(answers))) != 0:
#             if flag == 0:
#                 print(head, set(G.neighbors(head)), set(head_nbhood))
#             else:
#                 print(head, set(G.neighbors(head)), set(head_nbhood))
#                 break
    print(count)
#     print(sg, count)

  0%|          | 0/1 [00:00<?, ?it/s]

78


In [13]:
set(G.neighbors('Helen Mack'))

{'Divorce', 'Kiss and Make-Up', 'The Son of Kong'}

In [14]:
q_file_name = os.path.join(folder, 'MetaQA_half', 'qa_train_2hop.txt')
train_ha_pairs = readQuestions(q_file_name)
q_file_name = os.path.join(folder, 'MetaQA_half', 'qa_test_2hop.txt')
test_ha_pairs = readQuestions(q_file_name)

In [15]:
len(train_ha_pairs)

118980

In [16]:
len(test_ha_pairs)

14872

In [17]:
from collections import defaultdict
train_ha_dict = defaultdict(list)
for h,a in train_ha_pairs:
    train_ha_dict[h].append(set(a))

In [18]:
train_ha_dict['ginger rogers']

[]

In [19]:
count = 0
for h, a in tqdm(test_ha_pairs):
    if h not in train_ha_dict:
        continue
    answers_set_list = train_ha_dict[h]
    a = set(a)
    for ans_from_train in answers_set_list:
        if a == ans_from_train:
            count += 1
            break

  0%|          | 0/14872 [00:00<?, ?it/s]

In [20]:
count

2684

In [21]:
# get templates for questions
# then from templates get reasoning paths
# need to do this to get number of answerable questions for 2,3 hop
# just seeing reachability not enough for 2,3 hop
import copy

def split_qtype(qtype):
    qtype = qtype.split('_to_')
    return qtype

def relation_from_ent_type(ent):
    switcher = {
        'movie': 'error',
        'director': 'directed by',
        'actor': 'starred actors',
        'genre': 'has genre',
        'language': 'in language',
        'tags': 'has tags',
        'writer': 'written by',
        'year': 'release year',
        'tag': 'has tags',
    }
    return switcher.get(ent, 'error')

def qtype_to_path(qtype):
    sq = split_qtype(qtype)
    if len(sq) < 2:
        return []
    path = []
    for i in range(len(sq) - 1):
        start_ent = copy.copy(sq[i])
        end_ent = copy.copy(sq[i+1])
        if start_ent == 'movie':
            is_reverse = False
        else:
            is_reverse = True
            start_ent, end_ent = end_ent, start_ent
        rel = relation_from_ent_type(end_ent)
        if is_reverse:
            rel += ' reverse'
        path.append(rel)
    return path
        
def follow_path_1hop(head, graph, rel):
    return graph[head][rel]

# follow doesn't return head!
# this is ok since for a question, head never the answer
def follow_path(head, graph, path):
    ans_ents = []
    inter_ents = set([head])
    for rel in path:
        ents_for_next_step = set()
        for e in inter_ents:
            ents_1hop = follow_path_1hop(e, graph, rel)
            ents_for_next_step.update(ents_1hop)
        inter_ents = list(set(ents_for_next_step))
    out = set(inter_ents)
    if head in out:
        out.remove(head)
    return out

In [22]:
questions_folder = 'data/MetaQA_half'
train[0]

['The Prowler', 'starred actors', 'Evelyn Keyes']

In [23]:
len(getEntities(train)), len(train), len(G.edges)

(43234, 133582, 124680)

In [26]:
train[0]

['The Prowler', 'starred actors', 'Evelyn Keyes']

In [27]:
fname = os.path.join('data/MetaQA', 'train.txt')
full_triples = readTriples(fname)
entities = getEntities(full_triples)
len(entities)

43234

In [28]:
triples_file = 'data/MetaQA/train.txt'
train = readTriples(triples_file)
len(train)

133582

In [31]:
relations = set()
for t in full_triples:
    relations.add(t[1])
len(relations)

9

In [32]:
# manually make relational graph
# graph is a dict(dict(list))
from collections import defaultdict
graph = dict()
triples_for_graph = train
for e in entities:
    e = unidecode(e).rstrip()
    graph[e] = dict()
    for r in relations:
        graph[e][r] = set()
        graph[e][r + ' reverse'] = set()

for t in triples_for_graph:
    head, rel, tail = t
    rel_reverse = rel + ' reverse'
    graph[head][rel].add(tail)
    graph[tail][rel_reverse].add(head)
    

In [33]:
len(getEntities(triples_for_graph))

43234

In [34]:
qtype_folder = 'data/metaqa_qtype'
hops = '2'
qtype_split = 'test'
qtype_file = os.path.join(qtype_folder, hops+'-hop', 'qa_'+ qtype_split + '_qtype.txt')
qtypes = readLines(qtype_file)

q_file_name = os.path.join('data/MetaQA_half', 'qa_{}_{}hop.txt'.format(qtype_split, hops))
ha_pairs = readQuestions(q_file_name)

haq_triples = []
for i in range(len(ha_pairs)):
    item = (unidecode(ha_pairs[i][0]).rstrip(), ha_pairs[i][1], qtypes[i])
    haq_triples.append(item)

In [35]:
id = 0
haq_triples[id], qtype_to_path(haq_triples[id][2] )

(('John Krasinski',
  ['Nancy Meyers',
   'Sam Mendes',
   'George Clooney',
   'Ken Kwapis',
   'Luke Greenfield'],
  'actor_to_movie_to_director'),
 ['starred actors reverse', 'directed by'])

In [36]:
graph['John Krasinski']

{'has genre': set(),
 'has genre reverse': set(),
 'has imdb rating': set(),
 'has imdb rating reverse': set(),
 'starred actors': set(),
 'starred actors reverse': {'Away We Go',
  "It's Complicated",
  'Leatherheads',
  'License to Wed',
  'Nobody Walks',
  'Something Borrowed'},
 'has imdb votes': set(),
 'has imdb votes reverse': set(),
 'in language': set(),
 'in language reverse': set(),
 'has tags': set(),
 'has tags reverse': set(),
 'release year': set(),
 'release year reverse': set(),
 'written by': set(),
 'written by reverse': {'Brief Interviews with Hideous Men', 'Promised Land'},
 'directed by': set(),
 'directed by reverse': {'Brief Interviews with Hideous Men'}}

In [56]:
def get2hopPaths(head, tail):
    global relations, graph
    path_tail_pairs = []
    for key, value in graph[head].items():
        if len(value) > 0:
            for node in value:
                path_tail_pairs.append((node, key))
#     return path_tail_pairs
    output = []
    for mid, rel1 in path_tail_pairs:
        head_item = mid
        for rel2, v in graph[head_item].items():
            if len(v) == 0:
                continue
            for node in v:
                p = (node, [rel1, rel2])
                output.append(p)
    output_tail_filtered = []
    for o in output:
        if o[0] == tail:
            output_tail_filtered.append(o)
    return output_tail_filtered

def getPaths(head, tail, hops=1):
    global relations, graph
    

In [58]:
head = 'John Krasinski'
tail = 'Sam Mendes'
get2hopPaths(head, tail)

[('Sam Mendes', ['starred actors reverse', 'directed by'])]

In [68]:
from unidecode import unidecode

def getHeadEntityTemplate(s):
    index1 = s.find('[')
    index2 = s.find(']')
    head = s[index1 + 1: index2]
    qtext = s[:index1] + 'NE' + s[index2 + 1:]
    return head, qtext

def readQuestions(fname):
    f = open(fname)
    questions = []
    for line in f:
        line = line.rstrip().split('\t')
        qn = line[0]
        head, qtext = getHeadEntityTemplate(qn)
        answers = line[1].split('|')
        item = {}
        item['template'] = qtext
        item['head'] = unidecode(head).rstrip()
        item['answers'] = answers
        questions.append(item)
    f.close()
    return questions

fname = 'data/MetaQA_half/qa_train_2hop.txt'
questions = readQuestions(fname)

In [69]:
questions[10]

{'template': 'which movies share the same actor of NE',
 'head': 'The Strangers',
 'answers': ['Heavy',
  'Reign Over Me',
  'Underworld',
  'The Incredible Hulk',
  'Onegin',
  'Kitchen Party',
  'The Ledge',
  'My Life Without Me',
  "Cookie's Fortune",
  'The 24th Day',
  'Barefoot',
  'Armageddon',
  'Super']}

In [70]:
template_of_interest = 'which movies share the same actor of NE'
questions_filtered = []
for q in questions:
    if q['template'] == template_of_interest:
        questions_filtered.append(q)
len(questions_filtered)

963

In [80]:
from tqdm.notebook import tqdm
paths_counts = {}
for q in tqdm(questions_filtered):
    head = q['head']
    for tail in q['answers']:
        paths = get2hopPaths(head, tail)
        for path in paths:
            path = path[1]
            path_tuple = (path[0], path[1])
            if path_tuple in paths_counts:
                paths_counts[path_tuple] += 1
            else:
                paths_counts[path_tuple] = 1

  0%|          | 0/963 [00:00<?, ?it/s]

In [83]:
{k: v for k, v in sorted(paths_counts.items(), key=lambda item: item[1], reverse=True)}

{('starred actors', 'starred actors reverse'): 30895,
 ('has genre', 'has genre reverse'): 8143,
 ('has tags', 'has tags reverse'): 7541,
 ('release year', 'release year reverse'): 1048,
 ('starred actors', 'directed by reverse'): 666,
 ('written by', 'starred actors reverse'): 644,
 ('starred actors', 'written by reverse'): 625,
 ('directed by', 'directed by reverse'): 620,
 ('directed by', 'starred actors reverse'): 487,
 ('written by', 'written by reverse'): 425,
 ('in language', 'in language reverse'): 410,
 ('written by', 'directed by reverse'): 267,
 ('directed by', 'written by reverse'): 246,
 ('has imdb rating', 'has imdb rating reverse'): 58,
 ('has imdb votes', 'has imdb votes reverse'): 2,
 ('release year', 'has tags reverse'): 1,
 ('release year', 'has genre'): 1}

In [377]:
from collections import defaultdict
count = 0
qtype_answerable_dict = defaultdict(list)
for h, a, qt in haq_triples:
    path = qtype_to_path(qt)
    h = unidecode(h).rstrip()
    results = follow_path(h, graph, path)
    is_answerable = 0
    if len(set(results).intersection(set(a))) > 0:
        if len(set(results)) <= len(set(a)): # sanity check
            is_answerable = 1
            count += 1
    qtype_answerable_dict[qt].append(is_answerable)
#     if len(results) > 0:
#         count += 1
count, count/len(haq_triples)

(6333, 0.4436738125262715)

In [378]:
for k, v in qtype_answerable_dict.items():
    print(k, sum(v)/len(v))

movie_to_director_to_movie_to_language 0.17207792207792208
movie_to_director_to_movie_to_actor 0.372310570626754
movie_to_actor_to_movie_to_language 0.2857142857142857
movie_to_writer_to_movie_to_year 0.3085234093637455
movie_to_actor_to_movie_to_director 0.6492281303602059
movie_to_director_to_movie_to_genre 0.3712918660287081
movie_to_writer_to_movie_to_director 0.40458015267175573
movie_to_actor_to_movie_to_year 0.6254416961130742
movie_to_actor_to_movie_to_writer 0.6345177664974619
movie_to_actor_to_movie_to_genre 0.6524390243902439
movie_to_director_to_movie_to_writer 0.39183318853171156
movie_to_writer_to_movie_to_genre 0.4179640718562874
movie_to_writer_to_movie_to_actor 0.40778923253150057
movie_to_director_to_movie_to_year 0.32302092811646954
movie_to_writer_to_movie_to_language 0.266304347826087


In [208]:
count = 0
for h, a in ha_pairs:
    h = unidecode(h).rstrip()
    if h not in G:
        continue
    if len(set(G.neighbors(h)).intersection(set(a))) != 0:
        count += 1
count

6617

In [153]:
from unidecode import unidecode
head = 'Joe Thomas'
head = unidecode(head)
path = ['starred actors reverse']
follow_path(head, graph, path)

{'The Inbetweeners 2'}

In [150]:
graph[head]

{'has imdb rating': set(),
 'has imdb rating reverse': set(),
 'has imdb votes': set(),
 'has imdb votes reverse': set(),
 'has tags': set(),
 'has tags reverse': set(),
 'directed by': set(),
 'directed by reverse': set(),
 'has genre': set(),
 'has genre reverse': set(),
 'in language': set(),
 'in language reverse': set(),
 'release year': set(),
 'release year reverse': set(),
 'written by': set(),
 'written by reverse': set(),
 'starred actors': set(),
 'starred actors reverse': set()}

In [154]:
x = 'a'
y = 'b'

In [155]:
x, y

('a', 'b')

In [156]:
x, y = y, x
x, y

('b', 'a')